# Lambda-4S-5 Full Flight Simulation

Here I've created a model of the L-4S-5 flight. I want to see if this model matches published data so below are comparison charts to the source papers.


## Running The Simulation

In order to run this code you need to install a copy of [JSBSim](http://jsbsim.sourceforge.net/download.html)

Test to see if it's installed and in your path:

    $ JSBSim --version
    
      JSBSim Version: 1.0 Aug 14 2015 23:09:57

All of the analysis is in an [IPython Notebook](http://ipython.org/), though the maps and conversion utilties are in other packages. Make sure you have a modern juypiter based IPython

    $ sudo pip install ipython-notebook

And basemap plus data

    $ sudo apt-get install python-mpltoolkits.basemap python-mpltoolkits.basemap-data


To run the simulation, the post-procses analysis, and to genorate this document simply run:

    $ make


## Output

First  we start with an overview of the first three stage's altitude, velocity, and acceleration time histories:

In [ ]:
from numpy import loadtxt, array, arange, abs as npabs
from numpy.linalg import norm
import matplotlib.pyplot as plt
%matplotlib inline

columns = loadtxt("data.csv", delimiter=',', unpack=True, comments="T")

times              = columns[ 0]
MET                = columns[ 1]
altitudeMsl        = columns[ 2]
latitude           = columns[ 3]
longitude          = columns[ 4]
thrust             = columns[ 5] * 4.44822   # to newtons
total_mass         = columns[ 6] * 0.453592  # to kg
pitch              = columns[ 7]
ecei_vmag          = columns[ 8] * 0.3048    # to m/s
eci_vmag           = columns[ 9] * 0.3048    # to m/s
qbar               = columns[10] * 4.882e-4  # to kg/cm2
acc_u              = columns[11] * 0.3048    # to m/s2
acc_v              = columns[12] * 0.3048    # to m/s2
acc_w              = columns[13] * 0.3048    # to m/s2
downrange          = columns[14]
alpha              = columns[15]

accel = []
for i in range(len(acc_u)):
    accel.append(norm([acc_u[i], acc_v[i], acc_w[i]]))

fig, ax1 = plt.subplots(figsize=(16,9))
plt.title("L-4S-5 Flight Simulation Overview")
plt.xlabel(r"Time [s]")

ax1.set_ylabel(r"Acceleration [m/s$^2$]")
ax1.plot(MET, acc_u, color='#44bb55')
ax1.set_ylim([-50,250])
ax1.set_xlim([0,180])


ax2 = ax1.twinx()
#ax2.set_ylabel(r"Velocity [km/s]")
ax2.plot(MET, ecei_vmag/1000, color='#334455')
ax2.set_ylim([0,6])
ax2.set_xlim([0,180])
ax2.get_yaxis().set_tick_params(direction='in', pad=-15)

ax3 = ax2.twinx()
#ax3.set_ylabel(r"Dynamic Pressure [kg/cm$^2$]")
ax3.plot(MET, qbar, color='#bb44bb')
ax3.set_ylim([0,3])
ax3.set_xlim([0,180])
ax3.get_yaxis().set_tick_params(direction='in', pad=-25)

ax4 = ax1.twinx()
ax4.set_ylabel(r"Altitude [km]")
ax4.plot(MET, altitudeMsl/1000)
ax4.set_ylim([0,300])
ax4.set_xlim([0,180])



plt.show()

from IPython.display import Image
Image(filename='original-charts/L-4S-5-flightplan.png') 

## Downrange Charts

L-4S-5 was rail launched at an angle of 63&deg;. We want to look at the plots of Lambda as it travels downrange from the launch site.

### Pitch History

As the rocket flys it should gravity turn and we should see the pitch angle aproach 0.

In [ ]:
ax = plt.figure(figsize=(16,10))
plt.title(r"Pitch Time History")
plt.ylabel(r"Pitch [deg]")
plt.xlabel(r"Time [s]")
plt.plot(MET, pitch)
ax.axes[0].set_xlim([0,600])
ax.axes[0].set_ylim([0,80])
plt.show()

Image(filename='original-charts/L-4S-5-pitchplan.png') 

In [ ]:
ax = plt.figure(figsize=(16,10))
plt.title(r"Angle of Attack")
plt.ylabel(r"$\alpha$ [deg]")
plt.xlabel(r"Time [s]")
plt.plot(MET, alpha)
ax.axes[0].set_xlim([0,600])
ax.axes[0].set_ylim([-10,10])
plt.show()

### Altitude

The altitude as it flys downrange, roughly to scale:

In [ ]:
ax = plt.figure(figsize=(16,8))
plt.title(r"Downrange")
plt.ylabel(r"Altitude [km]")
plt.xlabel(r"Downrange [km]")
plt.plot(downrange/1000, altitudeMsl/1000)
plt.axis('equal')
plt.show()

## Launch Maps

Various maps of the launch

In [ ]:
from mpl_toolkits.basemap import Basemap
ax = plt.figure(figsize=(16,16))
plt.title("Launch Site Vicinity. First Stages")

m = Basemap(projection='aea',
            lat_0=31.2,lon_0=131.3,
            width=60e3,height=40e3,resolution ='h')

m.fillcontinents(color='#e7e7e7')
m.drawcoastlines(color='#dddddd')
m.drawcountries(color='#aaaaaa')
m.drawparallels(arange(30,33,0.1),labels=[1,1,0,0], color='#cccccc', dashes=(3,7))
m.drawmeridians(arange(129,135,0.1),labels=[0,0,0,1], color='#cccccc', dashes=(3,7))

x, y = m(longitude, latitude)
m.plot(x,y, color='r')


for i in [(8, "Booster Sep"), (32, "1st Stage Sep"), (37, "2nd Stage Ignition")]:
    idx = (npabs(MET-i[0])).argmin()
    mx, my = m(longitude[idx], latitude[idx])
    m.plot(mx, my, 'o', color="#ffcc22", markersize=4, markeredgewidth=0.1)
    plt.text(mx+50, my-130, i[1], color="#444444", rotation=45, ha='right', va='top')

for i in [(15, "0:15"), (30, "0:30"), (45, "0:45"), (60, "1:00")]:
    idx = (npabs(MET-i[0])).argmin()
    mx, my = m(longitude[idx], latitude[idx])
    m.plot(mx, my, 'o', color="#aaaaaa", markersize=2, markeredgewidth=0.1)
    plt.text(mx+0, my+150, i[1], color="#999999", rotation=45, ha='left', va='bottom')

plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
ax = plt.figure(figsize=(16,16))
plt.title("Medium Range. First 3 Stages And Coasting Arc")

m = Basemap(projection='aea',
            lat_0=31.0,lon_0=133.0,
            width=800e3,height=500e3,resolution ='h')

m.fillcontinents(color='#e7e7e7')
m.drawcoastlines(color='#dddddd')
m.drawcountries(color='#aaaaaa')
m.drawparallels(arange(25,45,1),labels=[1,1,0,0], color='#cccccc', dashes=(3,7))
m.drawmeridians(arange(125,140,1),labels=[0,0,0,1], color='#cccccc', dashes=(3,7))

lons = [130.55, 131.416667, 130.9587457]
lats = [  31.6,  31.916667, 30.3692168]
labels = ['Kagoshima City', 'Miyazaki', 'Tanegashima']
cx,cy = m(lons, lats)
m.plot(cx, cy, 'o', color="#aaaaaa", markersize=4)
for label, xpt, ypt in zip(labels, cx, cy):
    plt.text(xpt+3000, ypt+3000, label, color="#444444")


x, y = m(longitude, latitude)
m.plot(x,y, color='r')

for i in [(37, "2nd Stage Ignition"), (103, "3rd Stage Ignition"), (150, "3rd Stage Separation")]:
    idx = (npabs(MET-i[0])).argmin()
    mx, my = m(longitude[idx], latitude[idx])
    m.plot(mx, my, 'o', color="#ffcc22", markersize=4, markeredgewidth=0.1)
    plt.text(mx-1000, my-2000, i[1], color="#444444", rotation=45, ha='right', va='top')
    
for i in [(60, "1:00"), (60*2, "2:00"), (60*3, "3:00"), (60*4, "4:00")]:
    idx = (npabs(MET-i[0])).argmin()
    mx, my = m(longitude[idx], latitude[idx])
    m.plot(mx, my, 'o', color="#aaaaaa", markersize=2, markeredgewidth=0.1)
    plt.text(mx+0, my+5000, i[1], color="#999999", rotation=45, ha='left', va='bottom')


plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
ax = plt.figure(figsize=(16,16))
plt.title("Eastern Hemisphere Map")

m = Basemap(projection='ortho',
            lat_0=20.0,lon_0=180.0,resolution ='l')

m.fillcontinents(color='#e7e7e7')
m.drawcoastlines(color='#dddddd')
m.drawcountries(color='#aaaaaa')
m.drawparallels(arange(-60,60,10), color='#cccccc', dashes=(3,7))
m.drawmeridians(arange(-180,180,10), color='#cccccc', dashes=(3,7))



lons = [139.683333, 126.966667, -122.681944]
lats = [ 35.683333,  37.566667,   45.52]
labels = ['Tokyo', 'Seoul', 'Portland']
cx,cy = m(lons, lats)
m.plot(cx, cy, 'o', color="#aaaaaa", markersize=4)
for label, xpt, ypt in zip(labels, cx, cy):
    plt.text(xpt+20000, ypt+20000, label, color="#444444")
    
    
x, y = m(longitude, latitude)
m.plot(x,y, color='r')

plt.show()